<a href="https://colab.research.google.com/github/SJinLee/cryptology/blob/main/12.%EC%A0%84%EC%9E%90%EC%84%9C%EB%AA%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00


In [3]:
!pip list -v |grep pycryptodome

pycryptodome                     3.20.0                /usr/local/lib/python3.10/dist-packages pip


In [8]:
from Crypto import Random
from Crypto.Math.Numbers import Integer
from Crypto.Math.Primality import generate_probable_prime
randfunc = Random.get_random_bytes
bits = 3072
e = Integer(65537)

size_q = bits // 2
size_p = bits - size_q
min_p = min_q = (Integer(1) << (2 * size_q - 1)).sqrt()
def filter_p(candidate):
    return candidate > min_p and (candidate - 1).gcd(e) == 1
if size_q != size_p:
    min_p = (Integer(1) << (2 * size_p - 1)).sqrt()
p = generate_probable_prime(exact_bits=size_p,
                            randfunc=randfunc,
                            prime_filter=filter_p)

min_distance = Integer(1) << (bits // 2 - 100)

def filter_q(candidate):
    return (candidate > min_q and
            (candidate - 1).gcd(e) == 1 and
            abs(candidate - p) > min_distance)

q = generate_probable_prime(exact_bits=size_q,
                            randfunc=randfunc,
                            prime_filter=filter_q)

n = p * q
lcm = (p - 1).lcm(q - 1)
d = e.inverse(lcm)

if p > q:
    p, q = q, p

u = p.inverse(q)
{'n':n, 'e':e, 'd':d, 'p':p, 'q':q, 'u':u}

{'n': Integer(4383821717853748175865606638081124592782274595729185967809010355105900573602418913889458628895251505190834490398102491856097195019917465646986686231710675523751800649916029643407817531333231792922512554988290964941564307583956490968174858828215763928604776052516641415859010782391053098142143332887633551373161177209011197907480928100766648994147002778983148383152572019846786175130098129416712189634140004926086722675711868437589184706316611994939645742319314404944115859518686455692897965753061014018184209613465520622651540008575203105195731421306677773699977437852732933090107572051951832948165876602837305496535416591179585466813064421991334373956314228269429627607871129589115186074977207140723392588324019443431040641042790613173242160053158628765505384671865070001479528673702578230992226829326614073560577227722977069853441958454590907437796096272795599373878840688149251821933418776502086193739728839193316578496283),
 'e': Integer(65537),
 'd': Integer(1694912287682335141281

In [14]:
M = Integer(32)
S = pow(m,d,n)
M,S

(Integer(32),
 Integer(2907339436700559914558167920371940191575456548470107628421936036638357679464365826656381408952970749352233677734194809771758427655829845963566044297716576480649559728633934807142454047877995163765581757268902240007267048457651541630180853076889298095421110812827003172148745702963013316181004305133776323694889148323888900919261988765886102663376488947555855917587279666930238244520015549522106750667736555879146570423840940265336906859380901898355564288726213743935870171463147413338608052552484045595273065777766054446171663956615916006390955531373616376823698264849316282807848387719203798801049584356756081421297475396100481527847382155367519910392226727428388997003207305745658847310899956451286775276299051147312219638627013682996631179297950158443424461107527489335755903796317991349640038158249697504403338643104755355763778287163116545644009421003542499048424801854655175650120057934882042484849938078278709088623498))

In [17]:
pow(S,e,n) == M

True

# ElGamal

In [21]:
from Crypto.Math.Primality import generate_probable_safe_prime
bits = 180
p = generate_probable_safe_prime(exact_bits=bits, randfunc=randfunc)
q = (p - 1) >> 1

# Generate generator g
while 1:
    # Choose a square residue; it will generate a cyclic group of order q.
    g = pow(Integer.random_range(min_inclusive=2,
                                 max_exclusive=p,
                                 randfunc=randfunc), 2, p)

    # We must avoid g=2 because of Bleichenbacher's attack described
    # in "Generating ElGamal signatures without knowning the secret key",
    # 1996
    if g in (1, 2):
        continue

    # Discard g if it divides p-1 because of the attack described
    # in Note 11.67 (iii) in HAC
    if (p - 1) % g == 0:
        continue

    # g^{-1} must not divide p-1 because of Khadir's attack
    # described in "Conditions of the generator for forging ElGamal
    # signature", 2011
    ginv = g.inverse(p)
    if (p - 1) % ginv == 0:
        continue

    # Found
    break

# Generate private key x
x = Integer.random_range(min_inclusive=2,
                         max_exclusive=p-1,
                         randfunc=randfunc)
# Generate public key y
y = pow(g, x, p)
{'p':p,'g':g,'x':x,'y':y}


{'p': Integer(1112047258403358830383294653721914062049334603349413523),
 'g': Integer(85487169024922786654141015205878214126164586991912853),
 'x': Integer(20042234720261476519441772771653959824671447735279069),
 'y': Integer(490912430650429247988779818688483169830424961562077488)}

### encrypt, decrypt

In [22]:
r = Integer.random_range(min_inclusive=2,
                         max_exclusive=p-1,
                         randfunc=randfunc)
r

Integer(984509262666120594843065726317023088019949236134698029)

In [32]:
K = pow(y,r,p)
K

Integer(114199666640834114829681771050406370962308170128211597)

In [33]:
M=Integer(32)
C1 = pow(g,r,p)
C2 = pow(M*K,1,p)
C1,C2

(Integer(674769118023024794799287687476106197490892185058436294),
 Integer(318247557296615183399932712447261684645857634054530535))

In [34]:
pow(C1,x,p)

Integer(114199666640834114829681771050406370962308170128211597)

In [37]:
KI = K.inverse(p)
KI

Integer(807288064492619542347581940051148204401868213580412136)

In [38]:
pow(C2*KI,1,p)

Integer(32)

### 전자서명

In [48]:
while True:
    k = Integer.random_range(min_inclusive=2,
                            max_exclusive=p-1,
                            randfunc=randfunc)
    if k.gcd(p-1)==1:
        break

In [50]:
R = pow(g,k,p)

In [53]:
ki = k.inverse(p-1)

In [54]:
S = (M-x*R)*ki % (p-1)

In [55]:
pow(y,R,p)*pow(R,S,p) % p

Integer(471889376215868916105394608512200489615456936614001070)

In [56]:
pow(g,M,p)

Integer(471889376215868916105394608512200489615456936614001070)